<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div>

> **YugabyteDB DBA Fundamentals**
>
> Enroll for free  [Yugabyte University](https://university.yugabyte.com/).
>

## 🛠️ Requirements to run this locally
Here are the requirements to run this notebook locally:
- Download and install YugabyteDB
- Update the `env_vars.env` file
- Install Python 3.8 or higher
- 
### Download and install YugabyteDB
To run this notebook locally in VS Code on MacOs or Linux, you must download and install YugabyteDB:
#### MacOS 
You can download YugabyteDB for the MacOS at [https://download.yugabyte.com/#macos](https://download.yugabyte.com/#macos).

### Linux
You can download YugabyteDB for Linux at [https://download.yugabyte.com/#linux](https://download.yugabyte.com/#linux).

#### Docker
You can download YugabyteDB for Docker at [https://download.yugabyte.com/#docker](https://download.yugabyte.com/#docker).

## Update the `env_vars.env` file
After installing YugabyteDB locally, you need to update these paths in `env_vars.env`:
```
MY_YB_PATH_LOCAL_2.16=/Users/seth/yugabyte/yugabyte-2.16.3.0
MY_YB_PATH_LOCAL=/Users/seth/yugabyte/yugabyte-2.17.3.0
MY_YB_PATH_DATA_LOCAL=/Users/seth/yugabyte-data
```

### Install Python 3.8 or higher
For MacOs, PyEnv is probably the easiest way to install. You can learn more at:
- [https://github.com/pyenv/pyenv](https://github.com/pyenv/pyenv)
- [https://realpython.com/intro-to-pyenv/](https://realpython.com/intro-to-pyenv/)

---
## Update the notebook dependencies.

In [ ]:
!pip install --upgrade pip
!pip install ipython-sql~=0.5 --upgrade
!pip install psycopg2-binary
!pip install SQLAlchemy --upgrade 
!pip install sqlparse
!pip install html2text
!pip install jq

### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip editing and running this cell. 
> 

The following Python cell creates and stores the Jupyter notebook variables.
- First, edit the script to select how to assign a value to `MY_SUDO`.
- To run the script, select Execute Cell (Play Arrow in the cell side bar).
- Verify the accuracy of the output values

> About sudo and MacOS
> You'll need to supply your sudo password to create the loopback addresses.

In [ ]:
# Env variables for Notebook
import os

env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value
 
### BEGIN: Edit your MY_SUDO

# Leave it blank forcing a strategy for notebook execution
MY_SUDO=''

# env_vars defines ENV_SUDO='psswrd'
# MY_SUDO=os.environ.get('MY_SUDO')

# bash_profile exports the variable, e.g., export GLOBAL_SUDO='psswrd' 
MY_SUDO=os.environ.get('GLOBAL_SUDO')

### END: MY_SUDO


# Create and assign notebook variables
# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')
MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')

# Store the note book values for other notebooks to use
%store MY_SUDO
%store MY_DB_NAME
%store MY_YB_PATH
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_TSERVER_WEBSERVER_PORT

### Create the loopback host aliases for MacOS
> Important!
> 
> The script requires the password for `sudo`.
> 
> The bash cell utilizes the notebook variable, `MY_SUDO`.
> 
> If you did not define the `MY_SUDO` notebook variable using one of the suggested options, you can copy this script into the terminal, which is interactive, and manually supply the password.

The YugabyteDB local cluster for this lab requires that your create loop back aliases for your local host (127.0.0.1). The bash cell does the following:
- Deletes existing aliases
- Creates the loopback aliases from `127.0.0.2` to `127.0.0.7`

In [ ]:
%%bash -s "$MY_SUDO"  # ifconfig aliases
MY_SUDO=${1}

if ifconfig lo0 | grep 127.0.0.[2-7] > /dev/null
then
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.2
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.3
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.4
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.5
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.6
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.7
fi

echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.2
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.3
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.4
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.5
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.6
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.7

echo ${MY_SUDO} | sudo ifconfig lo0

### Create a YugabyteDB local cluster
- If a YugabyteDB local cluster is running, it will be stopped and destroyed using `yb-ctl`
- To avoid the port 9000 conflict, specifies port `TSERVER_WEBSERVER_PORT` as `8200`

In [21]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT" "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" # yugabyted
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}
HOST_IPv4_01=${3}
HOST_IPv4_02=${4}
HOST_IPv4_03=${5}

MASTER_FLAGS="yb_num_shards_per_tserver=1,ysql_beta_features=false,ysql_num_shards_per_tserver=1,num_cpu=2"
TSERVER_FLAGS="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=false,webserver_port="${TSERVER_WEBSERVER_PORT}",num_cpu=2"
echo $MASTER_FLAGS
echo $TSERVER_FLAGS
cd $YB_PATH

echo $YB_PATH
### Grep port 9000 for conflict
# lsof -nP -iTCP -sTCP:LISTEN | grep 9000

# Stop running cluster
if  pgrep -x "yb-tserver" > /dev/null 
then
    ./bin/yugabyted stop
    sleep 3
fi

# Destroy cluster
if echo `ps aux | grep yugabyted` | grep "yugabyted start"  > /dev/null 
then
    ./bin/yugabyted destroy
    sleep 3
fi



# Create cluster
./bin/yugabyted start \
--advertise_address=${HOST_IPv4_01} \
--base_dir=${YB_PATH}/node1 \
--cloud_location=cloud1.region1.zone1 \
--fault_tolerance=zone \
--ui=true \
--tserver_flags=""${TSERVER_FLAGS}"" \
--master_flags=""${MASTER_FLAGS}""


# Create cluster
./bin/yugabyted status
sleep 3

yb_num_shards_per_tserver=1,ysql_beta_features=false,ysql_num_shards_per_tserver=1,num_cpu=2
yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=false,webserver_port=8200,num_cpu=2
/Users/seth/yugabyte/yugabyte-2.19.0.0
Deleted logs at /Users/seth/var/logs.
Deleted data at /Users/seth/var/data.
Deleted conf file at /Users/seth/var/conf/yugabyted.conf.
yugabyted is not running.
Starting yugabyted...
✅ System checks
✅ YugabyteDB Started                  
✅ UI ready         
✅ Data placement constraint successfully verified                 

⚠ WARNINGS:
- Cluster started in an insecure mode without authentication and encryption enabled. For non-production use only, not to be used without firewalls blocking the internet traffic.


+----------------------------------------------------------------------------------------------------------+
|                                                yugabyted                                                 |
+-------------------

second step


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT" "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" # yugabyted
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}
HOST_IPv4_01=${3}
HOST_IPv4_02=${4}
HOST_IPv4_03=${5}

MASTER_FLAGS="yb_num_shards_per_tserver=1,ysql_beta_features=false,ysql_num_shards_per_tserver=1,num_cpu=2"
TSERVER_FLAGS="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=false,webserver_port="${TSERVER_WEBSERVER_PORT}",num_cpu=2"

cd $YB_PATH


### Grep port 9000 for conflict
# lsof -nP -iTCP -sTCP:LISTEN | grep 9000

# Stop running cluster
if  pgrep -x "yb-tserver" > /dev/null 
then
    ./bin/yugabyted stop
    sleep 3
fi

# Destroy cluster
if echo `ps aux | grep yugabyted` | grep "yugabyted start"  > /dev/null 
then
    ./bin/yugabyted destroy
    sleep 1
fi

# Create cluster
./bin/yugabyted start \
--advertise_address=${HOST_IPv4_01} \
--base_dir=${YB_PATH}/node1 \
--cloud_location=cloud1.region1.zone1 \
--fault_tolerance=zone \
--ui=true \
--tserver_flags=""${TSERVER_FLAGS}"" \
--master_flags=""${MASTER_FLAGS}""

sleep 15

./bin/yugabyted start \
--advertise_address=${HOST_IPv4_02} \
--join=${HOST_IPv4_01} \
--base_dir=${YB_PATH}/node2 \
--cloud_location=cloud2.region2.zone2 \
--fault_tolerance=zone \
--ui=true \
--tserver_flags ${TSERVER_FLAGS} \
--master_flags ${MASTER_FLAGS}

sleep 15

./bin/yugabyted start \
--advertise_address=${HOST_IPv4_03} \
--join=${HOST_IPv4_01} \
--base_dir=${YB_PATH}/node3 \
--cloud_location=cloud3.region3.zone3 \
--fault_tolerance=zone \
--ui=true \
--tserver_flags ${TSERVER_FLAGS} \
--master_flags ${MASTER_FLAGS}



## add node
## ./bin/yb-ctl add_node --placement_info "cloud1.region1.zone1" 

# Output status
./bin/yugabyted status

In the above, you'll see the status of the YugabyteDB cluster running locally.

### YCQL API

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH

# select version
./bin/ycqlsh -e "
  select cql_version from system.local;
  "  

### YSQL API

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# select version
./bin/ysqlsh -d yugabyte -c "
   select version();
"  

---
# 🌟 Well done! 
In this notebook, you setup a running YugabyteDB cluster locally on your MacOs. You'r ready to move to the Lab.